## onsite数据分析程序

### 配置文件
全局参数设置 **（必须执行）**

In [12]:
import sys
sys.path.append('../tester')
from connector import Connector

COMPETITIONS = ['2023031405', '2023031901', '2023031902', '2023031903']

# 数据库连接实例
database = Connector()

### 竞赛报名情况

#### 参与高校及团队文本数据

In [13]:
import pandas as pd

users = []
user_columns = database._get_col_name('user')

script = f"SELECT participantId, competitionId FROM entry WHERE competitionId in ({','.join(COMPETITIONS)})"
for participantId, competitionId in database._db_fetch(script):
    user = database._get_value(table='user', field='*', logic='AND', userId=participantId)
    user = dict(zip(user_columns, user[0]))
    user['competitionId'] = competitionId
    users.append(user)

part_info = pd.DataFrame(users)

# 高校数
print(f"参赛高校数：{len(set(part_info['school']))-2}个") # 去除3个测试项(tj, 1-IDM, 2-Lattice)
# 团队数
print(f"参赛团队数：{len(set(part_info['supervisorName']))-2}支") # 去除2个测试项(1-IDM, 2-Lattice)
# 参赛人数(按每个团队6人计算)
group_info = {}
for competitionId, rows in part_info.groupby('competitionId'):
    competition_name = database._get_value(table='competition', field='competitionName', logic='AND', competitionId=competitionId)[0][0]
    for index, row in rows.iterrows():
        group_info.setdefault(competition_name, [])
        group_info[competition_name].append(row.supervisorName)
for k, v in group_info.items():
    group_info[k] = len(set(v)) - 2
    print(f"  {k}：{group_info[k]}队")
print(f"参赛人数：{sum(list(group_info.values()))*6}人次")

参赛高校数：40个
参赛团队数：101支
  第一届OnSite自动驾驶算法挑战赛（综合赛）：42队
  第一届OnSite自动驾驶算法挑战赛（高速路基本段专项赛）：63队
  第一届OnSite自动驾驶算法挑战赛（高速路汇入汇出区专项赛）：39队
  第一届OnSite自动驾驶算法挑战赛（交叉口专项赛）：40队
参赛人数：1104人次


#### 高校报名分布情况
> 输出文件：`高校报名分布情况.html`

In [11]:
import plotly
import plotly.graph_objects as go
import pandas as pd

users = []
user_columns = database._get_col_name('user')
script = f"SELECT participantId, competitionId FROM entry WHERE competitionId in ({','.join(COMPETITIONS)})"
for participantId, competitionId in database._db_fetch(script):
    user = database._get_value(table='user', field='*', logic='AND', userId=participantId)
    user = dict(zip(user_columns, user[0]))
    user['competitionId'] = competitionId
    users.append(user)
part_info = pd.DataFrame(users)

competition_map = dict(zip(COMPETITIONS, [database._get_value(table='competition', field='competitionName', logic='AND', competitionId=competitionId)[0][0] for competitionId in COMPETITIONS]))
school_info = {}
for school, school_rows in part_info.groupby('school'):
    if school not in ['1-IDM', '2-Lattice']:
        school_info[school] = dict(zip(competition_map.values(), [[]]*len(COMPETITIONS)))
        school_info[school]['全赛道'] = school_rows['supervisorName'].unique().tolist()
        for competitionId, competition_rows in school_rows.groupby('competitionId'):
            school_info[school][competition_map[competitionId]] = competition_rows['supervisorName'].unique().tolist()

fig = go.Figure()
school_names = list(school_info.keys())
competition_names = list(school_info[school_names[0]].keys())
for name in competition_names:
    y = []
    text = []
    if name != '全赛道':
        for k, v in school_info.items():
            y.append(len(v[name]))
            text.append(','.join(v[name]))
        fig.add_trace(go.Bar(x=school_names, y=y, name=name[19:-1], text=text, 
                                texttemplate=y,
                                textposition="inside",
                                textfont_color="white",
                                textfont_size=8,
                                ))
    else:
        for k, v in school_info.items():
            y.append(-len(v[name]))
            text.append(','.join(v[name]))
        fig.add_trace(go.Bar(x=school_names, y=y, name='队伍数量', text=text, 
                                texttemplate=[-i for i in y],
                                textposition="inside",
                                textfont_color="white",
                                textfont_size=8,
                            ))   
fig.update_layout(barmode='relative', 
                xaxis={'categoryorder':'min ascending', 'tickangle':45, 'showgrid':True},
                legend=dict(
                                x=0.85,
                                y=0.95,
                                bgcolor='rgba(255, 255, 255, 0)',
                                bordercolor='rgba(255, 255, 255, 0)',
                                # font=dict(size=16), 
                            ),
                plot_bgcolor='rgba(0,0,0,0)',
                # width=2000,
                # height=1400,
                margin=dict(l=80, r=120, t=30, b=120)
                )
fig.show()
# fig.write_image("./analyze_result/school_info.svg", engine="kaleido")
plotly.offline.plot(fig, filename='./result/html/高校报名分布情况.html')

'./result/html/高校报名分布情况.html'

### 提交情况

#### 提交次数及规控器数量
包含A、B卷全部提交数据

In [23]:
import pandas as pd

script = f"SELECT * FROM submit WHERE competitionId in ({','.join(COMPETITIONS)})"
submits = database._db_fetch(script)
submits_df = pd.DataFrame(submits, columns=database._get_col_name('submit'))

print(f"共收到提交{len(submits_df)}次")
for competitionId, rows in submits_df.groupby('competitionId'):
    competition_name = database._get_value(table='competition', field='competitionName', logic='AND', competitionId=competitionId)[0][0]
    print(f"  {competition_name}：{len(rows)}次")

print(f"其中包含不同提交内容{len(set(submits_df[submits_df['status'] == 'SUCCESS']['dockerId'].to_list()))}种")

共收到提交693次
  第一届OnSite自动驾驶算法挑战赛（综合赛）：121次
  第一届OnSite自动驾驶算法挑战赛（高速路基本段专项赛）：339次
  第一届OnSite自动驾驶算法挑战赛（高速路汇入汇出区专项赛）：87次
  第一届OnSite自动驾驶算法挑战赛（交叉口专项赛）：146次
其中包含不同提交内容350种


#### B卷提交累加曲线
> 输出文件：`提交累加曲线.html`

In [22]:
import plotly
import plotly.graph_objects as go
import pandas as pd
import datetime

week_num = 11

script = f"SELECT * FROM submit WHERE competitionId in ({','.join(COMPETITIONS)})"
submits = database._db_fetch(script)
submits_df = pd.DataFrame(submits, columns=database._get_col_name('submit'))

group_submit_date = dict(zip(COMPETITIONS, [[0]*week_num, [0]*week_num, [0]*week_num, [0]*week_num]))
for competitionId, rows in submits_df.groupby('competitionId'):
    for index, row in rows.iterrows():
        try:
            group_submit_date[competitionId][row.submitTime.isocalendar()[1]-12] += 1
        except Exception as e:
            print(name, row.submitTime.isocalendar())
            
fig = go.Figure()
for k, v in group_submit_date.items():
    for i in range(1, len(v)):
        v[i] += v[i-1]
    fig.add_trace(go.Scatter(x=[f'第{i+1}周'for i in range(week_num)], y=v,
                mode='lines+markers',
                name=database._get_value(table='competition', field='competitionName', logic='AND', competitionId=k)[0][0][19:-1],))

fig.update_layout(legend=dict(
                                x=0.02,
                                y=0.98,
                                bgcolor='rgba(255, 255, 255, 0)',
                                bordercolor='rgba(255, 255, 255, 0)'
                            ),
                )
      
fig.show()
plotly.offline.plot(fig, filename='./result/html/提交累加曲线.html')

'./result/html/提交累加曲线.html'

### 测试情况

#### 各赛道有效测试数据
*有效测试数据*：B卷测试成功的提交

In [19]:
import pandas as pd
import json

def convert_time(time_str):
    # time_str: '00:00:00' 转换为秒
    h, m, s = time_str.split(':')
    return int(h) * 3600 + int(m) * 60 + int(s)

def cal_avg_time(time_list):
    avg = sum(time_list) / len(time_list)
    h = int(avg / 3600)
    m = int((avg - h * 3600) / 60)
    s = int(avg - h * 3600 - m * 60)
    return f"{h:02d}:{m:02d}:{s:02d}"

def get_max_time(time_list):
    mt = max(time_list)
    h = int(mt / 3600)
    m = int((mt - h * 3600) / 60)
    s = int(mt - h * 3600 - m * 60)
    return f"{h:02d}:{m:02d}:{s:02d}"

scirpt = f"SELECT * FROM submit WHERE competitionId IN ({', '.join(COMPETITIONS)}) AND paperType='B' AND status='SUCCESS'"
tests = database._db_fetch(scirpt)
tests_df = pd.DataFrame(tests, columns=database.columns)

for competitionId, rows in tests_df.groupby('competitionId'):
    costs = []
    competition_name = database._get_value(table='competition', field='competitionName', logic='AND', competitionId=competitionId)[0][0]
    print(competition_name)
    print(f"  有效测试数：{len(rows)}")
    print(f"  测试最高分：{rows['score'].max():.2f}")
    print(f"  测试平均分：{rows['score'].mean():.2f}")
    for index, row in rows.iterrows():
        time_use = json.loads(row.testDetail)['test']['cost']
        costs.append(float(time_use))
    print(f"  平均测试时长：{cal_avg_time(costs)}")
    print(f"  最长测试时长：{get_max_time(costs)}")

第一届OnSite自动驾驶算法挑战赛（综合赛）
  有效测试数：72
  测试最高分：93.61
  测试平均分：63.24
  平均测试时长：19:20:06
  最长测试时长：68:03:05
第一届OnSite自动驾驶算法挑战赛（高速路基本段专项赛）
  有效测试数：203
  测试最高分：93.42
  测试平均分：77.16
  平均测试时长：06:59:43
  最长测试时长：41:22:16
第一届OnSite自动驾驶算法挑战赛（高速路汇入汇出区专项赛）
  有效测试数：65
  测试最高分：96.70
  测试平均分：79.94
  平均测试时长：07:21:33
  最长测试时长：49:51:30
第一届OnSite自动驾驶算法挑战赛（交叉口专项赛）
  有效测试数：93
  测试最高分：94.61
  测试平均分：70.52
  平均测试时长：01:48:51
  最长测试时长：41:19:07


#### 各赛道测试得分分布

> 输出文件：`测试得分分布.html`

In [32]:
import plotly
import plotly.graph_objects as go
import pandas as pd

scirpt = f"SELECT * FROM submit WHERE competitionId IN ({', '.join(COMPETITIONS)}) AND paperType='B' AND status='SUCCESS'"
tests = database._db_fetch(scirpt)
tests_df = pd.DataFrame(tests, columns=database.columns)

fig = go.Figure()
for competitionId, rows in tests_df.groupby('competitionId'):
    competition_name = database._get_value(table='competition', field='competitionName', logic='AND', competitionId=competitionId)[0][0]
    fig.add_trace(go.Box(
                    y=rows['score'].tolist(),
                    name=competition_name[19:-1],
                    boxpoints=False,
                ))
fig.update_layout(
    yaxis=dict(
        range=[0, 100],
        tickmode='array',
        tickvals=[i*10 for i in range(11)],
        tickfont=dict(size=16),
    ),
    xaxis=dict(tickfont=dict(size=22)),
)
fig.show()
plotly.offline.plot(fig, filename='./result/html/测试得分分布.html')

'./result/html/测试得分分布.html'

#### 各赛道测试时长分布
> 输出文件：`测试时长分布.html`

In [28]:
import plotly
import plotly.graph_objects as go
import pandas as pd
import json

def convert_time(time_str):
    h, m, s = time_str.split(':')
    return int(h) * 3600 + int(m) * 60 + int(s)

def cal_avg_time(time_list):
    avg = sum(time_list) / len(time_list)
    h = int(avg / 3600)
    m = int((avg - h * 3600) / 60)
    s = int(avg - h * 3600 - m * 60)
    return f"{h:02d}:{m:02d}:{s:02d}"

scirpt = f"SELECT * FROM submit WHERE competitionId IN ({', '.join(COMPETITIONS)}) AND paperType='B' AND status='SUCCESS'"
tests = database._db_fetch(scirpt)
tests_df = pd.DataFrame(tests, columns=database.columns)

fig = go.Figure()
for competitionId, rows in tests_df.groupby('competitionId'):
    competition_name = database._get_value(table='competition', field='competitionName', logic='AND', competitionId=competitionId)[0][0]
    fig.add_trace(go.Box(
                        x=[float(json.loads(test_info)['test']['cost']) for test_info in rows.testDetail],
                        name=competition_name[19:-1],
                        # boxpoints="suspectedoutliers",
                        # boxpoints=False,
                        boxpoints="all",
                    ))
fig.update_layout(
    xaxis=dict(
        range=[0, 180000],
        tickmode='array',
        tickvals=[i*3600 for i in range(0, 60, 1)],
        ticktext=[f"{i}" for i in range(0, 60, 1)],
        tickfont=dict(size=16),
        title='测试时长(小时)',
    ),
    yaxis=dict(tickfont=dict(size=20)),
)
fig.show()
# fig.write_image("./analyze_result/test_cost.svg", engine="kaleido")
plotly.offline.plot(fig, filename='./result/html/测试时长分布.html')

'./result/html/测试时长分布.html'

#### 测试超时情况统计
> 输出文件：`测试超时名单.csv`

In [34]:
import json
import pandas as pd

competition_map = dict(zip(COMPETITIONS, [database._get_value(table='competition', field='competitionName', logic='AND', competitionId=competitionId)[0][0] for competitionId in COMPETITIONS]))

def convert_time(test_time):
    h = int(test_time / 3600)
    m = int((test_time - h * 3600) / 60)
    s = int(test_time - h * 3600 - m * 60)
    return f"{h:02d}:{m:02d}:{s:02d}"

def get_user_info(submitterId):
    return database._get_value(table='user', field='userName, mobile, email', logic='AND', userId=submitterId)[0]

def is_best(submitterId, competitionId, cur_score):
    scores = []
    if cur_score:
        for score in database._get_value(table='submit', field='score', logic='AND', submitterId=submitterId, competitionId=competitionId):
            if score[0]:
                scores.append(score[0])
        return max(scores) == cur_score
    else:
        return False

scirpt = f"SELECT * FROM submit WHERE competitionId IN ({', '.join(COMPETITIONS)}) AND paperType='B' AND status='SUCCESS'"
tests = database._db_fetch(scirpt)

timeouts = []
for submit in database._db_fetch(scirpt):
    try:
        test_detail = json.loads(submit[-1])
        test_time = float(test_detail["test"]["cost"])
    except:
        continue
    if submit[2] == '2023031405':
        # if test_time >= 3600 * 30 and is_best(submit[1], submit[2], submit[5]):
        if test_time >= 3600 * 30:
            userName, mobile, email = get_user_info(submit[1])
            timeouts.append([submit[0], submit[3], competition_map[submit[2]], userName, mobile, email, convert_time(test_time), submit[5], is_best(submit[1], submit[2], submit[5])])
    elif submit[2] == '2023031901' or submit[2] == '2023031902' or submit[2] == '2023031903':
        # if test_time >= 3600 * 10 and is_best(submit[1], submit[2], submit[5]):
        if test_time >= 3600 * 10:
            userName, mobile, email = get_user_info(submit[1])
            timeouts.append([submit[0], submit[3], competition_map[submit[2]], userName, mobile, email, convert_time(test_time), submit[5], is_best(submit[1], submit[2], submit[5])])

pd.DataFrame(timeouts, columns=['submitId', '提交时间', '赛道', '用户', '手机号', '邮箱', '测试耗时', '得分', '是否为本人最高分']).to_csv('./result/csv/测试超时名单.csv', index=False, encoding='gbk')

### 获奖队伍词云生成

> 输入文件: tester/tools/test_C中生成的`leaderboard_final.xlsx`

> 输出文件: parser/result/word_cloud中的png文件

In [2]:
from wordcloud import WordCloud
import pandas as pd
import imageio

mask_path = r"/home/ubuntu/onsite-test-server/parser/src/wordcloud_mask_car.png"
font_path = r"/home/ubuntu/onsite-test-server/parser/src/shangshoubaihushufati.ttf"
leaderboard_path = r"/home/ubuntu/onsite-test-server/tester/tools/test_C/leaderboard_final.xlsx"

leaderboard = pd.read_excel(leaderboard_path, sheet_name=None)

texts = []
for competitionName, rows in leaderboard.items():
    texts.extend(rows.head(10)['学校'].tolist())

texts = list(set(texts))
texts.extend(['同济大学']*2)
texts.extend(['清华大学'])
texts.extend(['长安大学'])
texts.extend(['东南大学'])
texts.extend(['浙江大学'])
text = ' '.join(texts)

# 创建词云对象
wordcloud = WordCloud(background_color='white', font_path=font_path, mask=imageio.imread(mask_path),
                      max_font_size=130, min_font_size=40, colormap='bone').generate(text)

# 将词云保存为图片文件
wordcloud.to_file('./result/word_cloud/my_wordcloud_9.png')

/tmp/ipykernel_1380028/2651595557.py:24: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  wordcloud = WordCloud(background_color='white', font_path=font_path, mask=imageio.imread(mask_path),


In [9]:
import pandas as pd
leaderboard_path = r"/home/ubuntu/onsite-test-server/tester/tools/test_C/leaderboard_final.xlsx"

leaderboard = pd.read_excel(leaderboard_path, sheet_name=None)

school_info = {}

for competitionName, rows in leaderboard.items():
    for index, row in rows.head(10).iterrows():
        school_info.setdefault(row['学校'], 0)
        school_info[row.学校] += 1
school_info_df = pd.DataFrame(zip(school_info.keys(), school_info.values()), columns=['学校', '入围次数'])
school_info_df.sort_values(by='入围次数', ascending=False, inplace=True)
school_info_df.to_csv('./result/word_cloud/高校前十入围次数.csv', index=False, encoding='utf-8')

### 各赛道排行榜变化趋势

#### 用户排行及分数变化
> 输出文件：`./result/leaderboard_change/{赛道名}_排行榜变化.csv`

In [5]:
import datetime
import pandas as pd

def get_date_list():
    dates = []
    start_date = datetime.date(2023, 4, 1)
    end_date = datetime.date(2023, 5, 31)
    delta = datetime.timedelta(days=1)
    current_date = start_date
    while current_date <= end_date:
        dates.append(current_date.strftime("%Y-%m-%d"))
        current_date += delta
    return dates

date_list = get_date_list()
    
scirpt = f"SELECT * FROM submit WHERE competitionId IN ({', '.join(COMPETITIONS)}) AND paperType='B' AND status='SUCCESS'"
tests = database._db_fetch(scirpt)
tests_df = pd.DataFrame(tests, columns=database.columns)

for competitionId, competition_rows in tests_df.groupby('competitionId'):
    leaderboard_change = {}
    competition_name = database._get_value(table='competition', field='competitionName', logic='AND', competitionId=competitionId)[0][0]
    competition_rows.dropna(subset=['score'], inplace=True)
    competition_rows.sort_values(by='submitTime', inplace=True, ascending=True)
    for userId, user_rows in competition_rows.groupby('submitterId'):
        user_name = database._get_value(table='user', field='userName', logic='AND', userId=userId)[0][0]
        leaderboard_change[user_name] = [0] * len(date_list)
        user_rows['score'] = user_rows['score'].cummax()
        user_rows['submitTime'] = user_rows['submitTime'].dt.strftime('%Y-%m-%d')
        user_rows.drop_duplicates(subset=['submitTime'], inplace=True, keep='last')
        last_pos = 0
        last_score = 0
        for index, row in user_rows.iterrows():
            pos = date_list.index(row['submitTime'])
            if last_pos < pos:
                delta = (row['score'] - last_score) / (pos - last_pos)
                for i in range(last_pos+1, pos+1):
                    last_score += delta
                    leaderboard_change[user_name][i] = last_score
                last_pos = pos
            else:
                last_score = row['score']
                leaderboard_change[user_name][pos] = last_score
            # print(pos, leaderboard_change[user_name][pos], row['score'])
        if last_pos != len(date_list) - 1:
            for i in range(last_pos+1, len(date_list)):
                leaderboard_change[user_name][i] = last_score
    
    user_change = []
    for user_name, scores in leaderboard_change.items():
        user_change.append([user_name, *scores])
    # print(pd.DataFrame(user_change, columns=['userName', *date_list]))
    user_change_df = pd.DataFrame(user_change, columns=['userName', *date_list]).round(4)
    user_change_df.to_csv(f'./result/leaderboard_change/{competition_name[19:-1]}_排行榜变化.csv', index=False, encoding='utf-8')

#### 各赛道前十平均分变化趋势
> 输出文件：`./result/leaderboard_change/排行榜前十平均分变化.csv`

In [30]:
import os
import pandas as pd

file_dir = r"./result/leaderboard_change/"

def get_date_list():
    dates = []
    start_date = datetime.date(2023, 4, 1)
    end_date = datetime.date(2023, 5, 31)
    delta = datetime.timedelta(days=1)
    current_date = start_date
    while current_date <= end_date:
        dates.append(current_date.strftime("%Y-%m-%d"))
        current_date += delta
    return dates

date_list = get_date_list()

new_df = pd.DataFrame()

for competitionId in COMPETITIONS:
    competitionName = database._get_value(table='competition', field='competitionName', logic='AND', competitionId=competitionId)[0][0]
    df = pd.read_csv(os.path.join(file_dir, f'{competitionName[19:-1]}_排行榜变化.csv'), encoding='utf-8')
    df.drop(columns=['userName'], inplace=True)
    top_mean = df.apply(lambda x: x.nlargest(10).mean(), axis=0)
    top_mean_df = pd.DataFrame([competitionName[19:-1], *top_mean]).T
    # print(top_mean_df)
    new_df = pd.concat([new_df, top_mean_df], axis=0, ignore_index=True)
    
new_df.columns = ['比赛名称', *date_list]
new_df.to_csv('./result/leaderboard_change/排行榜前十平均分变化.csv', index=False, encoding='utf-8')

#### 前三名用户提交次数及分数提升

In [29]:
import pandas as pd
leaderboard_path = r"/home/ubuntu/onsite-test-server/tester/tools/test_C/leaderboard_final.xlsx"

leaderboard = pd.read_excel(leaderboard_path, sheet_name=None)

for competitionName, competition_rows in leaderboard.items():
    competitionId = database._get_value(table='competition', field='competitionId', logic='AND', competitionName=competitionName)[0][0]
    print(competitionName)
    for index, user_row in competition_rows.head(10).iterrows():
        userName = user_row['主用户名']
        userId = database._get_value(table='user', field='userId', logic='AND', userName=userName)[0][0]
        user_submits = database._get_value(table='submit', field='*', logic='AND', competitionId=competitionId, submitterId=userId, status='SUCCESS', paperType='B')
        user_submits_df = pd.DataFrame(user_submits, columns=database.columns)
        # user_submits_df.dropna(subset=['score'], inplace=True)
        # user_submits_df.sort_values(by='submitTime', inplace=True, ascending=True)
        print((user_submits_df['score'].max()-user_submits_df['score'].min())/user_submits_df['dockerId'].nunique())
user_submits_df

第一届OnSite自动驾驶算法挑战赛（交叉口专项赛）
23.12856675
4.752499999999998
2.5462500000000006
9.0097
2.370766666666666
2.1303000000000005
0.192499999999999
4.6898333333333335
3.0201047619047623
3.1420250000000003
第一届OnSite自动驾驶算法挑战赛（综合赛）
0.0
9.177475
25.91965
9.57783333333333
3.2907636363636357
0.0
0.0
0.34779999999999944
8.0414
3.1971166666666675
第一届OnSite自动驾驶算法挑战赛（高速路基本段专项赛）
0.567025000000001
1.4562428571428572
0.160420000000002
0.7569000000000017
0.0
1.6675999999999993
5.721725000000003
1.724474074074074
0.9974857142857141
0.2292500000000004
第一届OnSite自动驾驶算法挑战赛（高速路汇入汇出区专项赛）
0.2874833333333342
0.0920000000000023
6.227099999999993
1.1987714285714293
1.4486500000000007
0.0
0.5465333333333314
0.0
3.076599999999999
0.8526333333333298


,submitId,submitterId,competitionId,submitTime,testTime,score,dockerId,paperType,resultLink,status,testDetail
0,s_20230529234746_20230427032549,20230427032549,2023031902,2023-05-29 23:47:46,2023-05-30 09:30:35,NaN,lvan1949/transformer_trajectory_planning:0.0.2,B,https://resource.onsite.run/results/2023031902...,SUCCESS,"{""pid"": 1633257, ""pull"": {""cost"": ""0.89"", ""sta..."
1,s_20230530215214_20230427032549,20230427032549,2023031902,2023-05-30 21:52:15,2023-06-04 10:02:17,65.8435,lvan1949/transformer_trajectory_planning:0.0.3,B,https://resource.onsite.run/results/2023031902...,SUCCESS,"{""pid"": 1633266, ""pull"": {""cost"": ""1.32"", ""sta..."
2,s_20230531223833_20230427032549,20230427032549,2023031902,2023-05-31 22:38:34,2023-06-04 09:59:37,68.4014,lvan1949/transformer_trajectory_planning:0.0.5,B,https://resource.onsite.run/results/2023031902...,SUCCESS,"{""pid"": 1633259, ""pull"": {""cost"": ""4.23"", ""sta..."
